In [22]:
import sqlite3

In [23]:
connect = sqlite3.connect('database.sqlite')
cur = connect.cursor()

# Task1.1

In [29]:
# go through each table in the database
for table in tables:
    print(f"Table: {table}")
    
    # pull all rows into a DataFrame
    data = pd.read_sql(f"SELECT * FROM {table}", connect)
    
    # build a quick profile of the columns
    details = pd.DataFrame({
        "Column": data.columns,
        "Type": [data[col].dtype for col in data.columns],
        "Filled": [data[col].count() for col in data.columns],
        "Missing": [data[col].isnull().sum() for col in data.columns],
        "Unique": [data[col].nunique() for col in data.columns]
    })
    
    display(details)
    print()


Table: follows


,Column,Type,Filled,Missing,Unique
0,follower_id,int64,7225,0,199
1,followed_id,int64,7225,0,210



Table: users


,Column,Type,Filled,Missing,Unique
0,id,int64,210,0,210
1,username,object,210,0,204
2,location,object,210,0,88
3,birthdate,object,210,0,199
4,created_at,object,210,0,183
5,profile,object,202,8,201
6,password,object,210,0,210



Table: sqlite_sequence


...

,Column,Type,Filled,Missing,Unique
0,id,int64,8276,0,8276
1,post_id,int64,8276,0,1213
2,user_id,int64,8276,0,151
3,reaction_type,object,8276,0,6



Table: comments


,Column,Type,Filled,Missing,Unique
0,id,int64,5804,0,5804
1,post_id,int64,5804,0,1215
2,user_id,int64,5804,0,153
3,content,object,5804,0,5749
4,created_at,object,5804,0,5802



Table: posts


,Column,Type,Filled,Missing,Unique
0,id,int64,1303,0,1303
1,user_id,int64,1303,0,155
2,content,object,1303,0,1226
3,created_at,object,1303,0,1283


In [31]:
# open a cursor and list all tables in the database
cursor = connect.cursor()
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()


In [32]:
import pandas as pd  

# check a few tables and preview their data
tables_to_check = ["reactions", "comments", "posts"]

for tbl in tables_to_check:
    data = pd.read_sql(f"SELECT * FROM {tbl}", connect)
    print(f"\nShowing first rows from: {tbl}")
    print(data.head())



Showing first rows from: reactions
   id  post_id  user_id reaction_type
0   1     2631       60          like
1   2     2631      146          like
2   3     2631       98          like
3   4     2631       43          like
4   5     2570       71          like

Showing first rows from: comments
   id  post_id  user_id                                            content  \
0   1     1963       55  Haha, I bet your neighbors are either loving o...   
1   2     1963       55  LOL, I'm imagining you as a mad scientist, sur...   
2   3     1963       55  Haha, I can just picture you rocking out in yo...   
3   4     1796       78  Are you seriously buying into this mindfulness...   
4   5     1796       13  This is great! Mindfulness can truly change pe...   

            created_at  
0  2022-12-04 02:36:15  
1  2022-12-02 20:16:20  
2  2022-12-03 02:54:02  
3  2022-12-07 06:35:14  
4  2022-12-02 17:44:02  

Showing first rows from: posts
     id  user_id                                  

# Task 1.2

In [36]:
# get all user ids
all_users = pd.read_sql("SELECT id FROM users", connect)

# get users who have posted, commented, or reacted
active_posters = pd.read_sql("SELECT DISTINCT user_id FROM posts", connect)
active_commenters = pd.read_sql("SELECT DISTINCT user_id FROM comments", connect)
active_reactors = pd.read_sql("SELECT DISTINCT user_id FROM reactions", connect)

# find users who never engaged in any activity
inactive_users = all_users[
    ~all_users["id"].isin(active_posters["user_id"]) &
    ~all_users["id"].isin(active_commenters["user_id"]) &
    ~all_users["id"].isin(active_reactors["user_id"])
]

print("Number of lurkers:", len(inactive_users))


Number of lurkers: 55


# Task 1.3

In [38]:

# query: sum up comments + reactions on each user's posts, rank top 5
sql = """
SELECT 
    u.username,
    SUM(
        (SELECT COUNT(*) FROM comments c WHERE c.post_id = p.id) +
        (SELECT COUNT(*) FROM reactions r WHERE r.post_id = p.id)
    ) AS engagement_score
FROM users u
JOIN posts p ON p.user_id = u.id
GROUP BY u.id, u.username
ORDER BY engagement_score DESC
LIMIT 5;
"""

# run and fetch results
influencers = pd.read_sql(sql, conn)

print("Top 5 users with the most engagement on their posts:")
print(influencers[["username", "engagement_score"]])



Top 5 users with the most engagement on their posts:
       username  engagement_score
0    WinterWolf               446
1   PinkPanther               386
2     PinkPetal               383
3  GoldenDreams               366
4     WildHorse               353


# Task 1.4

In [45]:


conn = sqlite3.connect("database.sqlite")
#SQL: find users who posted the same text at least 3 times (in posts or comments)
sql = """
WITH all_text AS (
    SELECT user_id, content FROM posts WHERE content IS NOT NULL
    UNION ALL
    SELECT user_id, content FROM comments WHERE content IS NOT NULL
),
text_counts AS (
    SELECT user_id, content, COUNT(*) AS times_used
    FROM all_text
    GROUP BY user_id, content
    HAVING COUNT(*) >= 3
)
SELECT u.username, t.content, t.times_used
FROM text_counts t
JOIN users u ON u.id = t.user_id
ORDER BY t.user_id, t.times_used DESC;
"""

# run query and bring results into a DataFrame
spamCheck = pd.read_sql(sql, conn)

print("Users who repeated the same content 3+ times:")
display(spamCheck)



Users who repeated the same content 3+ times:


,username,content,times_used
0,night_owl,? Mega Giveaway Alert! ? Follow all accounts w...,8
1,night_owl,?FLASH GIVEAWAY? Click the link in our bio to ...,7
2,night_owl,?FREE VACATION? Tag a friend you’d take to Bal...,4
3,night_owl,Find out why everyone is switching to this new...,4
4,night_owl,This one trick will make you $500/day from hom...,3
5,history_buff,A lot of you asked what helped me drop 5kg in ...,5
6,history_buff,Best way to clean your sneakers ? snag yours h...,5
7,history_buff,Mood: me refreshing for likes every 30 seconds...,5
8,history_buff,What do you think? #thoughts #motivationmonday...,4
9,history_buff,You need this travel pillow in your life ? sho...,3
